In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

from models.birefnet import BiRefNet
from utils import check_state_dict

c:\Users\ZODNGUY1\packages\micromamba\envs\segment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
birefnet = BiRefNet(bb_pretrained=False)
state_dict = torch.load("weights/BiRefNet-general-epoch_244.pth", map_location='cpu')
state_dict = check_state_dict(state_dict)
birefnet.load_state_dict(state_dict)

torch.set_float32_matmul_precision(['high', 'highest'][0])
birefnet.to('cuda')
birefnet.eval()
birefnet.half()
print("Model loaded")

C:\Users\ZODNGUY1\AppData\Local\Temp\ipykernel_26528\4263500741.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("weights/BiRefNet-general-epoch_2

Model loaded


In [7]:
def extract_object(birefnet, imagepath):
    # Data settings
    image_size = (1024, 1024)
    transform_image = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(imagepath)
    input_images = transform_image(image).unsqueeze(0).to('cuda').half()

    # Prediction
    with torch.no_grad():
        preds = birefnet(input_images)[-1].sigmoid().cpu()
    pred = preds[0].squeeze()
    pred_pil = transforms.ToPILImage()(pred)
    mask = pred_pil.resize(image.size)
    image.putalpha(mask)
    return image, mask

In [8]:
from pathlib import Path
import os

img_dir = Path("C:/Users/ZODNGUY1/datasets/zeiss/heart-bg/images")
out_dir = Path("C:/Users/ZODNGUY1/datasets/zeiss/heart/images")
os.makedirs(out_dir, exist_ok=True)

#### Export PNG

In [9]:
from tqdm import tqdm

for img_path in tqdm(img_dir.iterdir(), desc="Processing images"):
    image, _ = extract_object(birefnet, img_path)
    output_path = out_dir / img_path.name
    image.save(output_path)

Processing images: 565it [05:50,  1.61it/s]


#### Export mask

In [10]:
from tqdm import tqdm

for img_path in tqdm(img_dir.iterdir(), desc="Processing images"):
    image, mask = extract_object(birefnet, img_path)
    output_name = img_path.name.split('_')[1]
    output_path = out_dir / output_name
    mask.save(output_path)

Processing images: 446it [02:48,  2.64it/s]


#### Fix name

In [2]:
from pathlib import Path
import os

old_dir = Path("C:/Users/ZODNGUY1/datasets/zeiss/heart-transparent/images_old")
new_dir = Path("C:/Users/ZODNGUY1/datasets/zeiss/heart-transparent/images")
os.makedirs(new_dir, exist_ok=True)

In [4]:
from shutil import copy2
from tqdm import tqdm

for img_path in tqdm(old_dir.glob("*.png")):
    if img_path.stem.isdigit() and len(img_path.stem) == 3:
        new_name = f"{int(img_path.stem)}.png"
        new_path = new_dir / new_name
        copy2(img_path, new_path)

565it [00:01, 523.96it/s]
